<a href="https://colab.research.google.com/github/reidnersousa/Estudo/blob/cuda/SomaVetorialEx3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%%writefile somaVetorial.cu

#include<stdio.h>
#include<time.h>
#include<stdlib.h>

__global__ void FazTudoInt (float *v1, float *v2 ,float *v3 ,float *v4 )
{

	
  v2[blockIdx.x]=v2[blockIdx.x]+ v3[blockIdx.x];
  v4[blockIdx.x]=v2[blockIdx.x]* v4[blockIdx.x];
  v3[blockIdx.x]=v4[blockIdx.x]/ v3[blockIdx.x];
  v1[blockIdx.x]=v3[blockIdx.x]-50;
}


__global__ void add(int *a, int *b, int *soma) {
	soma[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];
}

__global__ void sub(int*a,int*b,int*sub){
  sub[blockIdx.x] = a[blockIdx.x] - b[blockIdx.x];
}



void random_ints(float *a, float N){
	int i;

	for (i=0;i < N; i++){
		a[i] = rand() % 100;
    printf("%3f ",a[i]);
	}
  puts("");
}



#define N 1000

int main(void) {
	float  *b, *c,*d , *soma ; // host copies of  b, c,d,soma
	float *d_a, *d_b, *d_c,*d_d; // device copies of a, b, c
	float *soma_d ; 
  float size = N * sizeof(float);
	float i;


	// Alloc space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);
  cudaMalloc((void **)&d_d, size);

  cudaMalloc((void **)&soma_d, size);


	srand(time(NULL));
	
	// Alloc space for host copies of a, b, c and setup input values
	b = (float *)malloc(size); random_ints(b, N);
  c = (float*)malloc(size) ; random_ints(c,N);
  d = (float*)malloc(size) ; random_ints(d,N);

	
  soma = (float *)malloc(size);
	
	
  
	// Copy inputs to device
	
	cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_d, d, size, cudaMemcpyHostToDevice);
	
	// Launch add() kernel on GPU with N blocks
	FazTudoInt<<<N,1>>>(d_a, d_b, d_c,d_d);
	
	// Copy result back to host
	cudaMemcpy(soma, d_a, size, cudaMemcpyDeviceToHost);
	
  

	for (i = 0.0; i< N-1.0; i++){
		printf(">>%f,",soma[i]); //'soma' e equivalente a 'a' no execicio 
	}
	printf("==%f\n",soma[i]);


	// Cleanup
	 free(b); free(c);free(d);
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c); cudaFree(d_d);
	
	
	return 0;
	}

Overwriting somaVetorial.cu


In [16]:
!nvcc somaVetorial.cu -o somaVetorial
!./somaVetorial

somaVetorial.cu(85): error: expression must have integral or unscoped enum type

somaVetorial.cu(87): error: expression must have integral or unscoped enum type

2 errors detected in the compilation of "somaVetorial.cu".
 21  49  66  86  95  93  73  15   5   6  35  96  37  93  91  28  54  40  56  71   4  13  94  76  75  21  57  55  18  15  63  91  65  29  77  60  22  51  75  80  57  10  28  46  56  20  74  10  12  83  34  16  96  28  92  72  49   2  79  20  17  43  11  82  24  89  94  99  40  69  79  49  32   7  47  40  79  73  50  92  56  36  60   5  65  53  29  14   7   8  34  24   3  46  59  28  87  53  27  79  75  58  80   7  17  27  47  97   0  49  41   9  38   1  14   3   6  43  69  13   3   4  90   7   2  49  87  41  54  14  20  29  24   0  88  41  79  87  38  79  89  31  40  27  85   6  82  91   1  51  57   5  55  99  64   9   0  51  50  54  17  22  36  41  22  76  34   1  16  25  33   5  56  25  84  41  32  66  85  33  17  94  90  25  93   6  34  93   9  37  99  26  59  87  19